# Peak Detection
This notebook is made for detecting peaks of the snippets extracted in module 2 using the function defined in x


In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pyarrow.feather as feather
from tqdm import tqdm

In [5]:
%run "3.0 - Functions_peakdetection.ipynb"

In [6]:
%run "4.0-HRV_Extraction_Function.ipynb"

In [7]:
#root = 'G:\My Drive\Molecool\Databases\Database1'
root = '/Volumes/GoogleDrive/.shortcut-targets-by-id/1sZgDE1M3o-bDINfAQYEXl_vqsKs03WxD/Molecool/Databases/Database1' # Jaume's directoru

In [21]:
patients = sorted(os.listdir(root))

for pat in tqdm(patients): #Looping throught the folder Sxxxx
    if not pat.endswith(('.json', 'ini','\r')):
        path_pat = root+'/'+pat
        folders = sorted(os.listdir(path_pat))
        for folder in folders:    #Looping through the different ecg types 
            if not folder.endswith('.ini') and folder != 'Icon\r':
                path_folder = path_pat + '/' +folder   
                snippets = sorted(os.listdir(path_folder))
                samplingRate = find_rate(path_folder)
                for snip_folder in snippets:  #Looping through the snippet folders
                    if not snip_folder.endswith(('.json', '.ftr', '.ini','\r')):
                        path_snip =path_folder + '/' + snip_folder
                        read_meta(path_snip)
                        read_peaks(path_snip, samplingRate)
                        read_hrv(path_snip, samplingRate)
                        #read_hrv_std(path_snip, samplingRate)
                        

100%|███████████████████████████████████████████| 56/56 [01:26<00:00,  1.54s/it]


In [15]:
def read_peaks(path,rate):
    """Function to open snippet files, get peaks, and write them to a new file"""
    df = pd.read_feather(path_snip + '/MSNIP.ftr')   #inputting the ecg feather files in a data frame
    df_peaks = _filteringdet(df['ecg_0'],rate)
    feather.write_feather(df_peaks, (path_snip + '/PEAKS.ftr'))

In [16]:
def read_meta(path):
    """Function that creates a new meta file for peak files"""
#     with open(path + '\ManualMeta.json') as json_file:
#         metaData = json.load(json_file)
#         json_file.close()
    data = {'Error Flag': False, 'Error Type': 'No error'}
    with open((path + '/PeakMeta.json'), "w") as outfile:
        json.dump(data, outfile)
        outfile.close()

In [17]:
def read_meta_clean(path,length):
    data = {'Error Flag': False, 'Length': length, 'Error Type': 'No error'}
    with open((path + '/CleanPeakMeta.json'), "w") as outfile:
        json.dump(data, outfile)
        outfile.close()

In [18]:
def find_rate(path):
    with open(path + '/Meta.json') as json_file:
        metaData = json.load(json_file)
        json_file.close()
        return int(metaData['Sampling rate'])

In [19]:
def read_hrv(path,rate):
    if path == root + "/S0250/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 45000 and value <= 220000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0273/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 50000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0273/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 0 and value <= 250000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0282/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 0 and value <= 100500:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0283/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 50000 and value <= 200000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0288/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 20000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0300/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 100000 and value <= 200400:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0310/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 100000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0317/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 50000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0318/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 0 and value <= 250000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0366/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 100000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0372/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 0 and value <= 250000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0381/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 0 and value <= 2300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0398/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 90000 and value <= 200000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0416/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 100000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0424/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 100000 and value <= 300000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0435/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 0 and value <= 250000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0435/holter/Snippet004":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        x_values = []
        rr_values = []
        for index,value in enumerate(df_hrv['x_values']):
            if value >= 25000 and value <= 200000:
                x_values.append(value)
                rr_values.append(df_hrv['R-R Interval Final'][index])
        df_clean['R-R Interval Final'] = rr_values
        df_clean['x_values'] = x_values
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0273/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0317/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0326/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0368/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0390/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0409/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0420/holter/Snippet004":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0423/holter/Snippet000":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0423/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0423/holter/Snippet003":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0423/holter/Snippet004":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0426/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0426/holter/Snippet003":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0426/holter/Snippet004":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0427/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0427/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0427/holter/Snippet004":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0430/holter/Snippet003":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0434/holter/Snippet001":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))
    if path == root + "/S0441/holter/Snippet002":
        df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
        df_hrv = RR_calculator(df)
        df_clean = pd.DataFrame()
        df_clean['R-R Interval Final'] = df_hrv['R-R Interval Final']
        df_clean['x_values'] = df_hrv['x_values']
        df_clean = _outlier_removal(df_clean)
        rr = df_clean['R-R Interval Final'].to_list()
        list_end = cos_correction(rr)
        df_clean['R-R Interval Final'] = list_end
        feather.write_feather(df_clean, (path_snip + '/clean_snippet.ftr'))
        read_meta_clean(path,len(df_clean['R-R Interval Final']))

In [20]:
def read_hrv_std(path,rate):
    df = pd.read_feather(path_snip + '/PEAKS.ftr')   #inputting the ecg feather files in a data frame
    df_hrv = RR_calculator(df)
    df_clean = pd.DataFrame()
    df_clean['R-R Interval Final'] = cos_correction(df_hrv['R-R Interval Final'])
    df_clean['x_values'] = df_hrv['x_values']
    return df_clean

In [3]:
# Check to see the state of the peaks
count = 0
patients = sorted(os.listdir(root))
for pat in tqdm(patients):     #Looping throught the folder Sxxxx
    if not pat.endswith(('.json', 'ini','\r')):
        path_pat = root+'/'+pat
        folders = sorted(os.listdir(path_pat))
        for folder in folders:    #Looping through the different ecg types 
            if not folder.endswith('.ini') and folder != 'Icon\r':
                path_folder = path_pat + '/' +folder   
                snippets = sorted(os.listdir(path_folder))
                samplingRate = find_rate(path_folder)
                for snip_folder in snippets:  #Looping through the snippet folders
                    print("In snippet folder " + snip_folder)
                    if not snip_folder.endswith(('.json', '.ftr', '.ini','\r')):
                        path_snip =path_folder + '/' + snip_folder
                        df = pd.read_feather(path_snip + '/MSNIP.ftr')
                        df_signal = df['ecg_0']
                        df_peaks = pd.read_feather(path_snip + '/PEAKS.ftr')
                        
                        y_values = []
                        for point in df_peaks['x_values']:
                            if point >= 0 and point < 5000:
                                y_values.append(df_signal[point])
                    
                        #print(y_values)
                        #print(df_peaks[df_peaks['x_values'] < 5000])

                        plt.plot(df_signal[0:5000])
                        plt.title('ECG signal ' + pat + snip_folder)
                        plt.xlabel('Samples(n)')
                        plt.ylabel('Amplitude(mV)')
                        plt.scatter(df_peaks[df_peaks['x_values'] < 5000]['x_values'],y_values,marker='o',c='red')
                        plt.show()
                        
                        plt.plot(df_signal)
                        plt.title('ECG signal ' + pat + snip_folder)
                        plt.xlabel('Samples(n)')
                        plt.ylabel('Amplitude(mV)')
                        plt.scatter(df_peaks['x_values'],df_peaks['y_values'],marker='o',c='red')
                        plt.show()
                        count += 1
                        
print(count)

NameError: name 'os' is not defined

## Checking snippets for accurate peak detection

In [ ]:
%matplotlib notebook

In [ ]:
path_snip = root + "\\S0256\\holter\\Snippet000"
df = pd.read_feather(path_snip + '/MSNIP.ftr')
df_signal = df['ecg_0']
df_peaks = pd.read_feather(path_snip + '/PEAKS.ftr')
                        
                          
plt.plot(df_signal)
plt.title('ECG signal ')
plt.xlabel('Samples(n)')
plt.ylabel('Amplitude(mV)')
plt.scatter(df_peaks['x_values'],df_peaks['y_values'],marker='o',c='red')
plt.show()

In [ ]:
# Definitely bad snippets
badApples = ["\\S0282\\holter\\Snippet002","\\S0296\\holter\\Snippet000","\\S0296\\holter\\Snippet001","\\S0296\\holter\\Snippet002","\\S0366\\holter\\Snippet001","\\S0366\\holter\\Snippet002","\\S0406\\holter\\Snippet001","\\S0406\\holter\\Snippet002","\\S0430\\holter\\Snippet000","\\S0432\\holter\\Snippet003"]

In [ ]:
# Flagging the snippets with bad peak detection
for snip in badApples:
    snip_path = root + snip
    
    data = {'Error Flag': True, 'Error Type': 'Badly detected peaks (through manual inspection)'}
        
    with open((snip_path + '/PeakMeta.json'), "w") as outfile:
        json.dump(data, outfile)
        outfile.close()
    